# Build FakeHuayi backend

## Basic Use

Required files:

- ./huayi_providers/fake_huayi/
    - \_\_init\_\_.py
    - fake_huayi.py
    - props_huayi.json
    - conf_huayi.json
    - defs_huayi.json (if Pulse Backend is applied, TODO)
- ./
    - qubits_data.csv
    - gates_data.csv

Import FakeHuayi backend

```
from huayi_providers.fake_huayi import *
FackHuayi()    # for V1 backend
FackHuayiV2()  # for V2 backend
```

Generate noise model
```
from qiskit_aer.noise.noise_model import NoiseModel
noise_Huayi = NoiseModel.from_backend(FakeHuayi())
```

Create .json files that required to build the backend

```
from huayi_providers.backend_build import build_from_file
c = build_from_file(backend_name="huayi",
                    backend_version="x.x.x",
                    qubits_data="qubits_data.csv",
                    gates_data="gates_data.csv")
```

## File Structure

#### Expriment data

**qubits_data.csv** contains the information of qubits, including
- T1 time (ms)
- T2 time (ms)
- frequency (MHz)
- readout error rate
- Probability of finding 0 when prepared in 1
- Probability of finding 1 when prepared in 0
- readout length (us)

All information should be accompanied with the measurement date and time.

**gates_data.csv** contains the information of gates, including
- qubits
- gate type
- error rate
- length
- gate name (optional)

The gate error and length are measured from experiment, and should be accompanied with the measurement date and time.

#### Dictionaries of the backend properties and configurations

**props_huayi.json**
```
{'backend_name': 'fakehuayi',
 'backend_version='0.0.1',
 'last_update_date': now_time(),
 'qubits': [q1, q2, ...],
 'gates': [g1, g2, ...],
 'general': []}
```
 qubit info (q):
```
 [{'date': ['T1_date'], 'name': 'T1', 'unit': 'ms', 'value': ['T1']},
  {'date': ['T2_date'], 'name': 'T2', 'unit': 'ms', 'value': ['T2']},
  {'date': ['frequency_date'], 'name': 'frequency', 'unit': 'MHz', 'value': ['frequency']},
  {'date': ['readout_error_date'], 'name': 'readout_error', 'unit': '', 'value': ['readout_error']},
  {'date': ['prob_meas0_prep1_date'], 'name': 'prob_meas0_prep1', 'unit': '', 'value': ['prob_meas0_prep1']},
  {'date': ['prob_meas1_prep0_date'], 'name': 'prob_meas1_prep0', 'unit': '', 'value': ['prob_meas1_prep0']},
  {'date': ['readout_length_date'], 'name': 'readout_length', 'unit': 'us', 'value': ['readout_length']}]
```
gate info (g):
```
{'qubits': ['qubits'],
 'gate': ['gate'],
 'parameters': [{'date': ['error_date'],
                 'name': 'gate_error',
                 'unit': '',
                 'value': ['gate_error']},
                {'date': ['length_date'],
                 'name': 'gate_length',
                 'unit': 'ms',
                 'value': ['gate_length']}],
 'name': ['name']}
```

**conf_huayi.json**

#### Dictionary of the Noise Model

The keys in NoiseModel are
- basis gates ({'id', 'x', 'sx', 'rz', 'cx'})
- noise instructions ({'id', 'x', 'sx', 'cx', 'measure', 'reset', ''})
- noise qubits ({0, ... , 26})
- default quantum errors (set None by default)
- default readout errors (set None by default)
- local quantum errors
- local readout errors
- custom noise passes (set None by default)

In [1]:
# Generate props_huayi.json from .csv experimental data
from huayi_providers.backend_build import build_from_file

c = build_from_file(backend_name='huayi',
       backend_version='0.0.1',
       qubits_data='qubits_data.csv', 
       gates_data='gates_data.csv')


Successfully created props_huayi.json
Successfully created conf_huayi.json
New backends created, please import the backends with:
from huayi_providers.fake_huayi import FakeHuayi, FakeHuayiV2


In [2]:
from huayi_providers.fake_huayi import FakeHuayi, FakeHuayiV2
from qiskit_aer.noise.noise_model import NoiseModel

print(FakeHuayi())
print(FakeHuayiV2())

noise_Huayi = NoiseModel.from_backend(FakeHuayi())

print(noise_Huayi.__dict__)

fake_huayi
{'_basis_gates': {'cz', 'rz', 'rx', 'reset', 'xy', 'id', 'z', 'ry', 'x'}, '_noise_instructions': {'cz', 'id', 'z', 'measure', 'x'}, '_noise_qubits': {0, 1, 2, 3, 4, 5, 6, 7}, '_default_quantum_errors': {}, '_local_quantum_errors': {'id': {(0,): QuantumError([(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CECA8815D0>, 0.9983580481106573), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CF0AEFEED0>, 8.48239690572117e-05), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CECA9E8B90>, 3.2245211237155924e-06), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CEC7BAC2D0>, 0.0005179221224597729), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CF0AF1E410>, 4.400446330123015e-08), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CF0AF1E5D0>, 1.6727974772894745e-09), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001CF0AF1EA50>, 0.0005179221224597729), (<qiskit.circuit.q

## Check the performance of the new backend

In [3]:
from qiskit.circuit.library import EfficientSU2
from qiskit import transpile
from qiskit import QuantumCircuit

n_qubits = 4
ansatz = EfficientSU2(n_qubits, 
                      su2_gates=['ry'], 
                      entanglement="linear", 
                      reps=1,
                      flatten=True)
print("Efficient SU2 ansatz")
print(ansatz.draw(fold=140, idle_wires=False))

c_huayi = transpile(ansatz, backend=FakeHuayi(), optimization_level=3)
print("Transpileed with Huayi backend, depth = {}".format(c_huayi.depth()))
print(c_huayi.draw(fold=140, idle_wires=False))


Efficient SU2 ansatz
     ┌──────────┐     ┌──────────┐                        
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[4]) ├────────────────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐            
q_1: ┤ Ry(θ[1]) ├┤ X ├─────■──────┤ Ry(θ[5]) ├────────────
     ├──────────┤└───┘   ┌─┴─┐    └──────────┘┌──────────┐
q_2: ┤ Ry(θ[2]) ├────────┤ X ├─────────■──────┤ Ry(θ[6]) ├
     ├──────────┤        └───┘       ┌─┴─┐    ├──────────┤
q_3: ┤ Ry(θ[3]) ├────────────────────┤ X ├────┤ Ry(θ[7]) ├
     └──────────┘                    └───┘    └──────────┘
Transpileed with Huayi backend, depth = 13
global phase: π
         ┌──────────┐┌─────────┐┌───────┐   ┌─────────┐ ┌───────┐   ┌──────────┐                                             
q_1 -> 1 ┤ Ry(θ[1]) ├┤ Ry(π/2) ├┤ Rx(π) ├─■─┤ Ry(π/2) ├─┤ Rx(π) ├─■─┤ Ry(θ[5]) ├─────────────────────────────────────────────
         ├──────────┤├─────────┤├───────┤ │ └─────────┘ └───────┘ │ ├─────────┬┘┌───────┐   ┌──────────┐                     
q_2 -> 3 ┤ 

I have manually forced the CZ gate error very large (~0.8) except for the couplings between [1,3], [1,5] and [3,7]
The transpiled circuit indeed only connects the low-error qubits

In [4]:
import pandas

pandas.read_csv('qubits_data.csv')

,T1,T1_date,T2,T2_date,frequency,frequency_date,readout_error,readout_error_date,prob_meas0_prep1,prob_meas0_prep1_date,prob_meas1_prep0,prob_meas1_prep0_date,readout_length,readout_length_date
0,52105.33738,2023-12-09T10:51+08:00,971.817532,2023-12-09T10:51+08:00,0.935122,2023-12-09T10:51+08:00,0.000618,2023-12-09T10:51+08:00,0.000012,2023-12-09T10:51+08:00,0.000424,2023-12-09T10:51+08:00,0.100699,2023-12-09T10:51+08:00
1,49240.82938,2023-12-09T10:51+08:00,942.204255,2023-12-09T10:51+08:00,1.011607,2023-12-09T10:51+08:00,0.000399,2023-12-09T10:51+08:00,0.000284,2023-12-09T10:51+08:00,0.000117,2023-12-09T10:51+08:00,0.089423,2023-12-09T10:51+08:00
2,50459.09178,2023-12-09T10:51+08:00,933.875260,2023-12-09T10:51+08:00,1.019855,2023-12-09T10:51+08:00,0.000133,2023-12-09T10:51+08:00,0.000134,2023-12-09T10:51+08:00,0.000121,2023-12-09T10:51+08:00,0.134335,2023-12-09T10:51+08:00
3,50262.81399,2023-12-09T10:51+08:00,948.760790,2023-12-09T10:51+08:00,0.977639,2023-12-09T10:51+08:00,0.000616,2023-12-09T10:51+08:00,0.000197,2023-12-09T10:51+08:00,0.000109,2023-12-09T10:51+08:00,0.119865,2023-12-09T10:51+08:00
4,49246.76211,2023-12-09T10:51+08:00,981.791771,2023-12-09T10:51+08:00,1.009949,2023-12-09T10:51+08:00,0.000336,2023-12-09T10:51+08:00,0.000005,2023-12-09T10:51+08:00,0.000096,2023-12-09T10:51+08:00,0.103291,2023-12-09T10:51+08:00
5,50206.23176,2023-12-09T10:51+08:00,1034.525822,2023-12-09T10:51+08:00,0.993678,2023-12-09T10:51+08:00,0.000475,2023-12-09T10:51+08:00,0.000010,2023-12-09T10:51+08:00,0.000189,2023-12-09T10:51+08:00,0.081938,2023-12-09T10:51+08:00
6,50834.08581,2023-12-09T10:51+08:00,936.641372,2023-12-09T10:51+08:00,0.931331,2023-12-09T10:51+08:00,0.000214,2023-12-09T10:51+08:00,0.000367,2023-12-09T10:51+08:00,0.000003,2023-12-09T10:51+08:00,0.111399,2023-12-09T10:51+08:00
7,50197.89568,2023-12-09T10:51+08:00,1088.049384,2023-12-09T10:51+08:00,0.937623,2023-12-09T10:51+08:00,0.000338,2023-12-09T10:51+08:00,0.000205,2023-12-09T10:51+08:00,0.000046,2023-12-09T10:51+08:00,0.101157,2023-12-09T10:51+08:00


In [5]:
pandas.read_csv('gates_data.csv')

,qubits,gate,gate_error,error_date,gate_length,length_date,name
0,[0],id,0.001094,2023-12-11T16:25+08:00,168.276518,2023-12-11T16:25+08:00,id0
1,[1],id,0.000310,2023-12-11T16:25+08:00,166.562209,2023-12-11T16:25+08:00,id1
2,[2],id,0.002184,2023-12-11T16:25+08:00,156.740126,2023-12-11T16:25+08:00,id2
3,[3],id,0.001676,2023-12-11T16:25+08:00,177.642326,2023-12-11T16:25+08:00,id3
4,[4],id,0.001426,2023-12-11T16:25+08:00,149.358827,2023-12-11T16:25+08:00,id4
...,...,...,...,...,...,...,...
75,"[7,2]",cz,0.800728,2023-12-11T16:40+08:00,130.446578,2023-12-11T16:40+08:00,cz7_2
76,"[7,3]",cz,0.000826,2023-12-11T16:40+08:00,146.528246,2023-12-11T16:40+08:00,cz7_3
77,"[7,4]",cz,0.800130,2023-12-11T16:40+08:00,145.679879,2023-12-11T16:40+08:00,cz7_4
78,"[7,5]",cz,0.800326,2023-12-11T16:40+08:00,126.877120,2023-12-11T16:40+08:00,cz7_5
